# Adaptive Hashing for IP Address Lookup in Computer Networks

In [73]:
import math
import csv
import operator as op
from functools import reduce
import random
import numpy 

In [74]:
def ip_to_binary(ip):
    return (''.join([bin(int(x)+256)[3:] for x in ip.split('.')]))

In [75]:
file = open("IP_Database/Artificial/AIP256.csv")
csvreader = csv.reader(file)
header = next(csvreader)
print(header)
db = []
for row in csvreader:
    db.append(row[0])
print(db)
file.close()

['ï»¿256 Artificially Generated Databases']
['83.118.34.209', '205.33.79.234', '132.107.46.66', '245.48.255.135', '152.136.114.185', '78.203.254.86', '59.196.190.243', '188.102.84.13', '230.152.117.234', '223.78.172.103', '42.92.232.27', '254.182.255.234', '90.216.208.156', '149.229.231.18', '10.236.170.97', '224.164.218.98', '118.113.205.38', '3.157.75.69', '250.196.81.50', '56.96.165.60', '174.129.25.212', '217.239.243.68', '191.249.62.54', '96.182.86.211', '252.194.18.7', '171.100.54.13', '199.157.116.231', '54.116.127.99', '61.134.115.233', '233.9.69.51', '203.36.126.249', '103.191.215.92', '28.153.56.136', '56.70.61.36', '5.66.32.53', '0.222.122.241', '103.115.111.154', '120.156.71.255', '182.25.206.21', '94.235.33.231', '151.134.115.45', '234.63.141.140', '51.162.216.88', '219.206.22.232', '5.136.162.45', '67.51.158.182', '6.23.170.165', '106.50.155.182', '103.112.6.96', '189.184.226.128', '97.87.11.55', '219.54.36.191', '46.188.154.0', '124.66.72.55', '225.34.228.155', '187.83.1

In [76]:
binary_ip = []
for ip in db:
    ip = ip_to_binary(ip)
    binary_ip.append(ip)   
# print(binary_ip)

In [77]:
M = len(binary_ip)
n = len(binary_ip[0])
m = int(math.log2(M))
print(M)
print(n)
print(m)

256
32
8


# Ad Hoc Pivot Hashing

In [78]:
def nCr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer//denom   

def round_to_nearest_even(f):
    up = math.ceil(f / 2.) * 2
    down = math.floor(f / 2.) * 2
    if(up-f > f-down):
        return down
    else:
        return up

def d_XORing(di,dj):
    xi = (M-di)//2
    xj = (M-dj)//2
    if(xi<xj):
        (xi,xj) = (xj,xi)
    delta = xi-xj
    decimal_xor_d = 0
    for k in range(0,xj+1):
        prob = (nCr(xi,xj-k)*nCr(M-xi,k))/nCr(M,xj)
        decimal_xor_d += (abs(M-2*delta-4*k)*prob)  
    even_xor_d =  round_to_nearest_even(decimal_xor_d)
    return [decimal_xor_d,even_xor_d]

def reduction_on_xor(di,dj):
    xor_d = d_XORing(di,dj)[0]
    min_d = min(di,dj)
    if(min_d==0):
        return -1e9
    pij = (min_d - xor_d)/min_d
    return pij

def next_avg_μ(d):
    d = sorted(d)
    μ_pre = 0
    for i in range(0,m):
        μ_pre += d[i]        
    μ_pre = μ_pre//m
    return μ_pre

def combined_values(i,j,d):
    if(i>=0 and j<len(d)):
        return [reduction_on_xor(d[i],d[j]),d_XORing(d[i],d[j])[1],i,j]
    return [-2e50,-1,i,j]


In [79]:
def adhoc_pivot():
    # initial calculation of d-values
    d = [0]*n
    for row in binary_ip:
        for col in range(0,n):
            d[col] += int(row[col])
    for col in range(0,n):
        d[col] = abs(2*d[col]-M)
    
    while(len(d)>m):
        
        # Sort intermediate hash values based on d
        d = sorted(d)
        
        # Find μ-pre
        μ_pre = next_avg_μ(d)
        
        # Find pivot point
        pivot_position = 0
        max_red = -1e9
        for i in range(0,len(d)):
            for j in range(i+1,len(d)):
                per_red = reduction_on_xor(d[i],d[j])
                if(per_red > max_red):
                    max_red = per_red
                    pivot_position = i if d[i] < d[j] else j
        
        combo1 = combined_values(pivot_position-1,pivot_position+1,d)
        combo2 = combined_values(pivot_position-1,pivot_position,d)
        combo3 = combined_values(pivot_position,pivot_position+1,d) 
        
        best_pair_pivot = max(combo1,combo2,combo3)
        
        d_pivot = d[:]
        d_pivot.pop(best_pair_pivot[3])
        d_pivot.pop(best_pair_pivot[2])
        d_pivot.append(best_pair_pivot[1])
        
        # Find the μ-pivot
        μ_pivot = next_avg_μ(d_pivot)
        
          
        # Find the μ-m+1
        combo4 = combined_values(m-1,m,d)
        combo5 = combined_values(m,m+1,d)
        
        best_pair = max(combo4,combo5)
        
        d_m_plus_1 = d[:]
        d_m_plus_1.pop(best_pair[3])
        d_m_plus_1.pop(best_pair[2])
        d_m_plus_1.append(best_pair[1])
        
        μ_m_plus_1 = next_avg_μ(d_m_plus_1)
        
        if(μ_m_plus_1 > μ_pre and μ_pivot > μ_pre):
            break
        elif(μ_m_plus_1 < μ_pivot):
            d = d_pivot[:]
        else:
            d = d_m_plus_1[:]
    
    # resultant d 
    return d
       

In [80]:
d = adhoc_pivot()
binary_hash_keys = []
l = len(d)
for i in range(0,l):
    if(random.randint(0,1)):
        one = (M+d[i])//2
        zero = M-one
        col = [0]*zero + [1]*one
        random.shuffle(col)
        binary_hash_keys.append(col)
    else:
        zero = (M+d[i])//2
        one = M-zero
        col = [0]*zero + [1]*one
        random.shuffle(col)
        binary_hash_keys.append(col)
        
binary_hash_keys = numpy.transpose(binary_hash_keys)
binary_hash_keys = binary_hash_keys.tolist()
hash_keys = []
for i in range(len(binary_hash_keys)):
    res = int("".join(str(x) for x in binary_hash_keys[i]), 2)
    hash_keys.append(res)

ip_key = {}
for i in range(M):
    ip_key[db[i]] = hash_keys[i]

ip_hashmap = {}
for i in range(M):
    ip_hashmap.setdefault(hash_keys[i], [])
for i in range(M):    
    ip_hashmap[hash_keys[i]].append(db[i])


def adhoc_pivot_search_ip(ip):
    collisions = 0
    if(ip_key[ip]):
        for curr_ip in ip_hashmap[ip_key[ip]]:
            if(curr_ip==ip):
                return [True,collisions]
            else:
                collisions += 1
    return [False,0]

hashed_values = [0]*M
asl = 0
look_up_count = 0
msl = 0
for i in range(M):
    look_up = adhoc_pivot_search_ip(db[i])
    if(look_up[0]):
        hashed_values[i] += look_up[1]
        msl = max(msl,hashed_values[i])
        look_up_count += 1
        asl += hashed_values[i]
asl = asl/look_up_count       
print([asl,msl])
# print(hashed_values)

[0.24609375, 3]


# Other Well-Known Hashing Algorithms

### 1) Linear Searching Algorithm for IP Address

In [81]:
def linear_search_ip(ip): 
    collisions = 0
    for curr_ip in db:
        if(curr_ip==ip):
            return [True,collisions]
        else:
            collisions += 1
    return [False,0]

linear_hashed_values = [0]*M
linear_asl = 0
linear_look_up_count = 0
linear_msl = 0
for i in range(M):
    linear_look_up = linear_search_ip(db[i])
    if(linear_look_up[0]):
        linear_hashed_values[i] += linear_look_up[1]
        linear_msl = max(linear_msl,linear_hashed_values[i])
        linear_look_up_count += 1
        linear_asl += linear_hashed_values[i]
linear_asl = linear_asl/linear_look_up_count       
print([linear_asl,linear_msl])

[127.5, 255]


### 2) Standard Group XORing Algorithm

In [82]:
def standard_group_XORing():
    group_size=n//m
    binary_grid_hashes=[]
    for i in range(M):
        row_hash=[]
        for j in range(m):
            curr=0
            for k in range(j*group_size,j*group_size+group_size):
                curr^=int(binary_ip[i][k])
            row_hash.append(curr)
        binary_grid_hashes.append(row_hash)
    grid_hashes=[]
    for i in range(len(binary_grid_hashes)):
        res = int("".join(str(x) for x in binary_grid_hashes[i]), 2)
        grid_hashes.append(res)
    return grid_hashes

group_XORing_keys=standard_group_XORing()

group_XORing_ip_hashmap = {}
for i in range(M):
    group_XORing_ip_hashmap.setdefault(group_XORing_keys[i], [])
for i in range(M):    
    group_XORing_ip_hashmap[group_XORing_keys[i]].append(db[i])

def group_XORing_search_ip(ip): 
    curr_hash=[]
    original_ip=ip
    ip=ip_to_binary(ip)
    group_size=n//m
    for j in range(m):
        curr=0
        for k in range(j*group_size,j*group_size+group_size):
            curr^=int(ip[k])
        curr_hash.append(curr)
        
    hashed_ip_key=int("".join(str(x) for x in curr_hash), 2)
    collisions = 0
    if(group_XORing_ip_hashmap[hashed_ip_key]):
        for curr_ip in group_XORing_ip_hashmap[hashed_ip_key]:
            if(curr_ip==original_ip):
                return [True,collisions]
            else:
                collisions += 1
        return [False,collisions]
    return [False,0]

group_XORing_hashed_values = [0]*M
group_XORing_asl = 0
group_XORing_look_up_count = 0
group_XORing_msl = 0
for i in range(M):
    group_XORing_look_up = group_XORing_search_ip(db[i])
    if(group_XORing_look_up[0]):
        group_XORing_hashed_values[i] += group_XORing_look_up[1]
        group_XORing_msl = max(group_XORing_msl,group_XORing_hashed_values[i])
        group_XORing_look_up_count += 1
        group_XORing_asl += group_XORing_hashed_values[i]
group_XORing_asl = group_XORing_asl/group_XORing_look_up_count       
print([group_XORing_asl,group_XORing_msl])

[0.48046875, 5]


### 3) Advanced Bit Extraction Algorithm

In [83]:
def bit_extraction():
    # calculation of d-values
    d = [0]*n
    for row in binary_ip:
        for col in range(0,n):
            d[col] += int(row[col])
    for col in range(0,n):
        d[col] = abs(2*d[col]-M)
    # Sort hash values based on d
    d = sorted(d)
    d=d[:m]
    return d

bit_extraction_d = bit_extraction()
bit_hash_keys = []
bit_extraction_l = len(bit_extraction_d)
for i in range(0,bit_extraction_l):
    if(random.randint(0,1)):
        one = (M+bit_extraction_d[i])//2
        zero = M-one
        col = [0]*zero + [1]*one
        random.shuffle(col)
        bit_hash_keys.append(col)
    else:
        zero = (M+bit_extraction_d[i])//2
        one = M-zero
        col = [0]*zero + [1]*one
        random.shuffle(col)
        bit_hash_keys.append(col)
        
bit_hash_keys = numpy.transpose(bit_hash_keys)
bit_hash_keys = bit_hash_keys.tolist()
bit_extraction_hash_keys = []
for i in range(len(bit_hash_keys)):
    res = int("".join(str(x) for x in bit_hash_keys[i]), 2)
    bit_extraction_hash_keys.append(res)

bit_extraction_ip_key = {}
for i in range(M):
    bit_extraction_ip_key[db[i]] = bit_extraction_hash_keys[i]

bit_extraction_ip_hashmap = {}
for i in range(M):
    bit_extraction_ip_hashmap.setdefault(bit_extraction_hash_keys[i], [])
for i in range(M):    
    bit_extraction_ip_hashmap[bit_extraction_hash_keys[i]].append(db[i])


def bit_extraction_search_ip(ip):
    collisions = 0
    if(bit_extraction_ip_key[ip]):
        for curr_ip in bit_extraction_ip_hashmap[bit_extraction_ip_key[ip]]:
            if(curr_ip==ip):
                return [True,collisions+len(bit_extraction_ip_hashmap[bit_extraction_ip_key[ip]])]
            else:
                collisions += 1
    return [False,0]

bit_hashed_values = [0]*M
bit_asl = 0
bit_look_up_count = 0
bit_msl = 0
for i in range(M):
    bit_look_up = bit_extraction_search_ip(db[i])
    if(bit_look_up[0]):
        bit_hashed_values[i] += bit_look_up[1]
        bit_msl = max(bit_msl,bit_hashed_values[i])
        bit_look_up_count += 1
        bit_asl += bit_hashed_values[i]
bit_asl = bit_asl/bit_look_up_count       
print([bit_asl,bit_msl])
# print(hashed_values)

[2.541176470588235, 9]


In [84]:
# List 
Result_List=["AIP256",asl,linear_asl,bit_asl,group_XORing_asl,msl,linear_msl,bit_msl,group_XORing_msl]
# Open our existing CSV file in append mode
# Create a file object for this file
with open('A_performance.csv', 'a') as f_object:
    
    # Pass this file object to csv.writer()
    # and get a writer object
    writer_object = csv.writer(f_object)
  
    # Pass the list as an argument into
    # the writerow()
    writer_object.writerow(Result_List)
  
    #Close the file object
    f_object.close()